# Import libraries

In [ ]:
pip install adjustText

In [ ]:
import requests
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoLocator
from scipy.stats import percentileofscore
from adjustText import adjust_text

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

# Get data

In [ ]:
# creating list of gameweeks
gameweeks = []
for x in range(39):
  gameweeks.append(x)

In [ ]:
# static store of .json for all league information
# The ‘bootstrap-static’ call returns a huge amouth of information, and is all you really need if you don’t want to get into specific FPL managed teams.

# Data included:

# A summary of all 38 gameweeks
# The game’s settings
# Basic information on all 20 PL teams
# Total number of FPL Users and overall chip usage
# Basic information on all Premier League players
# List of stats that FPL keeps track of
# The different FPL positions

static = requests.get("https://fantasy.premierleague.com/api/bootstrap-static/").json()

# Create tables

## Fixtures

In [ ]:
# get fixtures data

def get_fixtures(link): # this function takes a link and produces a dataframe
  fixtures = pd.DataFrame(requests.get(link).json())
  return fixtures

In [ ]:
# create fixtures dataframe
fixtures_link = "https://fantasy.premierleague.com/api/fixtures/"
fixtures = get_fixtures(fixtures_link)
fixtures.head()

## Standings

In [ ]:
# get standings data

def get_standings(league_id):
  link = "https://fantasy.premierleague.com/api/leagues-classic/" + str(league_id) + "/standings"
  standings = requests.get(link).json()
  standings_df = pd.DataFrame(standings['standings']['results'])[['entry','id','entry_name','player_name','event_total','total','rank','last_rank','rank_sort']]
  standings_df['point_differential'] = standings_df['total'] - standings_df['total'].max()
  standings_df['rank_change'] = standings_df['last_rank'] - standings_df['rank']
  team_ids = standings_df['entry'].tolist()
  return standings_df, team_ids

In [ ]:
# create standings dataframe and list of team_ids
league_id = 1028900
standings, team_ids = get_standings(league_id)

In [ ]:
standings.head()

## EPL Teams

In [ ]:
# get epl team data

def get_epl_teams(drop_columns, static):
  epl_teams = pd.DataFrame(static['teams']).drop(columns=drop_columns)
  return epl_teams

In [ ]:
# epl teams table
drop_columns=[
    'position',
    'team_division',
    'unavailable'
]
epl_teams = get_epl_teams(drop_columns, static)

In [ ]:
epl_teams.head()

## Gameweek Stats

In [ ]:
# extract team (entry) stats for each gameweek (event)

def get_gameweek_stats(team_ids):
  gw_stats = pd.DataFrame()
  for j in team_ids:
    df=pd.DataFrame(requests.get("https://fantasy.premierleague.com/api/entry/"+str(j)+"/history/").json()['current'])
    df['entry'] = j
    gw_stats=pd.concat([gw_stats, df], ignore_index=True)
  gw_stats['event'] = gw_stats['event'].astype(int)
  gw_stats['rank'] = gw_stats.groupby(['event'])['total_points'].rank(method='first',ascending=False).astype(int)
  gw_stats['value'] = gw_stats['value'].astype(int) / 10
  return gw_stats

In [ ]:
# create gw_stats dataframe for all results
gw_stats = get_gameweek_stats(team_ids)
gw_stats.head()

## Global FPL Table

In [ ]:
# weekly FPL totals frome static events
FPL_totals = pd.DataFrame(static['events']).drop(columns=[
    'deadline_time',
    'deadline_time_epoch',
    'deadline_time_game_offset',
    'h2h_ko_matches_created',
    'highest_scoring_entry',
    'is_previous',
    'is_current',
    'is_next',
    'cup_leagues_created'
])

# only keep weeks that are finished scoring
FPL_totals = FPL_totals[FPL_totals.finished==True]

# store the latest week global average score
FPL_avg_score = FPL_totals.iloc[-1]['average_entry_score']

In [ ]:
FPL_totals.tail()

## Position Key

In [ ]:
positions = pd.DataFrame(static['element_types']).drop(columns=[
    'plural_name',
    'plural_name_short',
    'ui_shirt_specific',
    'sub_positions_locked'
])

positions

## Squads

In [ ]:
def get_squads(team_ids, current_week):
  squads = pd.DataFrame()
  for i in range(1,current_week+1):
    for j in team_ids:
      x = requests.get("https://fantasy.premierleague.com/api/entry/"+str(j)+"/event/"+str(i)+"/picks/").json()
      df=pd.DataFrame(x['picks'])
      df['entry'] = j
      df['event'] = i
      squads=pd.concat([squads,df])
  return squads

In [ ]:
current_week = gw_stats['event'].max()

squads = get_squads(team_ids,current_week)

In [ ]:
squads.head()

## Weekly Player Stats

In [ ]:
def get_player_stats(current_week):
  player_stats = pd.DataFrame()
  for i in range(1,current_week+1):
    df=pd.DataFrame(requests.get("https://fantasy.premierleague.com/api/event/"+str(i)+"/live/").json())
    df['week'] = i
    for key in df['elements'][0].keys():
      df[key] = df['elements'].apply(lambda x: x[key])
    for key in df['elements'][0]['stats'].keys():
      df[key] = df['stats'].apply(lambda x: x[key])
    player_stats = pd.concat([player_stats,df])
  player_stats = player_stats.drop(columns=['elements','stats','explain'])
  return player_stats

In [ ]:
player_stats = get_player_stats(current_week)
player_stats.head()

## Player Summary

In [ ]:
# player stats and info
players = pd.DataFrame(static['elements'])

float_convert_list = [
    'selected_by_percent',
    'value_form',
    'value_season',
    'influence',
    'creativity',
    'threat',
    'ict_index',
    'form',
    'points_per_game',
    'expected_assists',
    'expected_goals',
    'expected_goal_involvements',
    'expected_goals_conceded'
]

for x in float_convert_list:
  players[x] = players[x].astype(float)

In [ ]:
players.head()

## Player Position Bridge

In [ ]:
player_positions = players[['element_type','id','first_name','second_name','web_name']].merge(positions[['id','singular_name']], how='inner',left_on='element_type',right_on='id').drop(columns=['element_type','id_y'])
player_positions.columns = ['player_id','first_name','second_name','web_name','position']
player_positions

# Visuals

## Standings

In [ ]:
data=standings[['rank','rank_change','entry_name', 'player_name','event_total','total','point_differential']]
data.columns = ['Place', 'Place Change','Team Name','Manager','Gameweek Points','Total Points','Total Point Differential']
data

## Standings by week

In [ ]:
data=pd.merge(
      gw_stats[['entry','event','rank']],
      standings[['entry','entry_name']],
      on='entry'
)

plt.figure(figsize=(10,5))

sns.lineplot(
    data=data,
    x='event',
    y='rank',
    hue='entry_name',
    markers=True,
    marker='s',
    markersize=10
)

plt.yticks([x for x in range(1,11)])
plt.xlabel('Gameweek')
plt.xticks(range(data['event'].min(), data['event'].max()+1))
plt.ylabel('Table Position')
plt.gca().invert_yaxis()
plt.title('Position by Gameweek')

plt.legend(
    loc='lower center',
    bbox_to_anchor=(0.5,-0.30),
    ncol=5
)

## Squad Value

In [ ]:
data = pd.merge(
      gw_stats[['entry','event','value']],
      standings[['entry','entry_name']],
      on='entry'
)

plt.figure(figsize=(10,5))

sns.lineplot(
    data=data,
    x='event',
    y='value',
    hue='entry_name',
    markers=True,
    marker='s',
    markersize=10
)

plt.xlabel('Gameweek')
plt.xticks(range(data['event'].min(), data['event'].max()+1))
plt.ylabel('Squad Value')
plt.title('Squad Value')

plt.legend(
    loc='lower center',
    bbox_to_anchor=(0.5,-0.30),
    ncol=5
)

## Points Behind 1st

In [ ]:
# get entry name on the gw_stats df
data=pd.merge(
      gw_stats[['entry','event','total_points']],
      standings[['entry','entry_name']],
      on='entry'
  )

# create a point differential from the leader each week by team
data['pt_diff'] = data['total_points'] - data.groupby(['event'])['total_points'].transform('max')
plt.figure(figsize=(10,5))

sns.lineplot(
    data=data,
    x='event',
    y='pt_diff',
    hue='entry_name',
    markers=True,
    marker='s',
    markersize=10
)

plt.xlabel('Gameweek')
plt.ylabel('Points from First')
plt.xticks(range(data['event'].min(), data['event'].max()+1))
plt.legend(
    loc='lower center',
    bbox_to_anchor=(0.5,-0.30),
    ncol=5
)

## Weekly Average Scores

In [ ]:
# adding entry name to gw_stats
data=pd.merge(
      gw_stats[['entry','event','points']],
      standings[['entry','entry_name']],
      on='entry'
  )

# creating df for weekly average global scores
data2=FPL_totals[['id','average_entry_score','name']]
data2.columns=['entry','points','entry_name']
data2['entry_name'] = 'FPL Average'

plt.figure(figsize=(10,5))

# plot gw_stat scores
sns.lineplot(
    data=data,
    x='event',
    y='points',
    hue='entry_name',
    markers=True,
    marker='s',
    markersize=10
)

# plot fpl weekly global average
sns.lineplot(
    x=data['event'],
    y=data2['points'],
    linestyle='--',
    marker='o',
    label='FPL Avg',
    color='black',
    markersize = 15,
    linewidth=2
)

plt.xlabel('Gameweek')
plt.ylabel('Points')
plt.xticks(range(data['event'].min(), data['event'].max()+1))
plt.legend(
    loc='lower center',
    bbox_to_anchor=(0.5,-0.30),
    ncol=5
)
plt.title('Weekly Points')

## TOTWs

In [ ]:
data = pd.merge(squads[['element','event','entry']], player_stats[player_stats.in_dreamteam == True][['week','id','in_dreamteam']], how='inner',left_on=['element','event'], right_on=['id','week'])
data = pd.merge(data[['week','entry','in_dreamteam']].groupby(['week','entry']).count().reset_index(), standings[['entry','entry_name']], how='inner', on='entry')
data = data.drop(columns=['entry'])
data.columns = ['week','totws','entry_name']

In [ ]:
plt.figure(figsize=(10,5))

sns.barplot(
    data = data[['entry_name','totws']].groupby('entry_name').sum().sort_values('totws', ascending=False).reset_index(),
    x='totws',
    y='entry_name'
)

plt.title('Team Of The Week Picks')
plt.xlabel('')
plt.ylabel('')

## Players Below FPL Global Average

In [ ]:
data = standings[standings.event_total < FPL_avg_score][['rank','entry_name', 'player_name','event_total']].sort_values('event_total', ascending=False)
data['diff'] = data['event_total'] - FPL_avg_score
data.columns = ['rank', 'team', 'manager', 'week_score', 'vs_avg']
data

## Team Score Boxplot

In [ ]:
data=pd.merge(
      gw_stats[['entry','event','points']],
      standings[['entry','entry_name']],
      on='entry'
  )

plt.figure(figsize=(10,5))

sns.boxplot(
    data=data,
    x='entry_name',
    y='points'
)

plt.xticks(rotation=20)
plt.xlabel('Team')
plt.ylabel('Points')

## Player Value

## Top Player Shortlist

using the following features to create the shortlist:

- form
  - goal involvements
  - xgi
  - clean sheets (defenders)
  - bonus points
- fixtures

### Fixture Difficulty

In [ ]:
a = players[['id','first_name','second_name','web_name','element_type','team','team_code','now_cost','selected_by_percent','form','value_season','value_form','total_points','bonus','minutes','goals_scored','assists','clean_sheets','goals_conceded','yellow_cards','saves','ict_index','starts','expected_goal_involvements','expected_goals_conceded','corners_and_indirect_freekicks_order','direct_freekicks_order','penalties_order','expected_goal_involvements_per_90','saves_per_90','expected_goals_conceded_per_90','goals_conceded_per_90','clean_sheets_per_90']]

players_df = pd.merge(
    a,
    positions[['id','singular_name']],
    left_on='element_type',
    right_on='id'
).drop(columns=['id_y']).rename(columns={'id_x':'id'})


fixtures_df = pd.merge(
    epl_teams[['code','id','name','strength','strength_overall_home','strength_overall_away','strength_attack_home','strength_attack_away','strength_defence_home','strength_defence_away']],
    fixtures[(fixtures.event>current_week)&(fixtures.event<=current_week+5)][['team_a','team_h','team_a_difficulty']].groupby('team_a').agg({'team_h':'count','team_a_difficulty':'sum'}).sort_values('team_a').reset_index(),
    left_on='id',
    right_on='team_a'
).drop(columns=['team_a']).rename(columns={'team_h':'away_games','team_a_difficulty':'away_difficulty'})

fixtures_df = fixtures_df.merge(
    fixtures[(fixtures.event>current_week)&(fixtures.event<=current_week+5)][['team_h','team_a','team_h_difficulty']].groupby('team_h').agg({'team_a':'count','team_h_difficulty':'sum'}).sort_values('team_h').reset_index(),
    left_on='id',
    right_on='team_h'
).drop(columns=['team_h']).rename(columns={'team_a':'home_games','team_h_difficulty':'home_difficulty'})

fixtures_df['total_games_next_five'] = fixtures_df['away_games'] + fixtures_df['home_games']
fixtures_df['total_difficulty_next_five'] = fixtures_df['away_difficulty'] + fixtures_df['home_difficulty']
fixtures_df['adj_away_difficulty'] = fixtures_df['away_difficulty'] * 1.22
fixtures_df['adj_total_difficulty_next_five'] = fixtures_df['adj_away_difficulty'] + fixtures_df['home_difficulty']

shortlist = pd.merge(
    players_df[['team','first_name','second_name','web_name','singular_name','now_cost','selected_by_percent','form','value_form','total_points','value_season','bonus','minutes','goals_scored','assists','clean_sheets','goals_conceded','yellow_cards','saves','ict_index','expected_goal_involvements','expected_goals_conceded','corners_and_indirect_freekicks_order','direct_freekicks_order','penalties_order']],
    fixtures_df[['id','name','strength','away_games','away_difficulty','home_games','home_difficulty','total_games_next_five','total_difficulty_next_five','adj_away_difficulty','adj_total_difficulty_next_five']],
    left_on='team',
    right_on='id'
).drop(columns=['team','id']).rename(columns={'name':'team_name'}).sort_values('total_points', ascending=False)

shortlist['goal_involvements'] = shortlist['goals_scored'] + shortlist['assists']

In [ ]:
data = fixtures_df[['name','strength','adj_total_difficulty_next_five']].sort_values('adj_total_difficulty_next_five')

plt.figure(figsize=(10,10))

sns.barplot(
    data=data,
    x='adj_total_difficulty_next_five',
    y='name',
    # color = 'blue',
    hue='strength',
    dodge=False,
    palette='tab10'
)

plt.title('Next Five Fixture Difficulty')
plt.xlabel('Adjusted Total Difficulty, Next Five Fictures')

## Top 30 Players

In [ ]:
df=shortlist[['web_name','singular_name','team_name','now_cost','total_points','form','adj_total_difficulty_next_five']].sort_values('form',ascending=False).head(30).reset_index()
df['now_cost'] = df['now_cost'] / 10
df.columns = ['index','name','position','team','cost','total_points','form','fixture_difficulty']
df.drop(columns='index')

In [ ]:
data = df[['position','name']].groupby('position').count().sort_values('name', ascending=False).reset_index()
data['pc'] = data['name'] / data['name'].sum()

sns.barplot(
    data=data,
    x='position',
    y='name'
)

plt.xlabel('Position')
plt.ylabel('# of Top 30')
plt.title('Top Players by Position')

In [ ]:
data=df[['position','total_points']].groupby('position').sum().sort_values('total_points',ascending=False).reset_index()
data['pc'] = data['total_points'] / data['total_points'].sum()
data

In [ ]:
sns.barplot(
    data=data,
    x='position',
    y='pc'
)

plt.title('Points from Positions')
plt.xlabel('Position')
plt.ylabel('% of All Points')

# Exploration

In [ ]:
selections = squads[['element','entry']].groupby('element').count().reset_index().sort_values('entry',ascending=False).merge(player_positions[['player_id','web_name','position']], how='inner', left_on='element', right_on='player_id')
selections = selections[['player_id','web_name','position','entry']]
selections['selections_percent'] = round(100 * selections['entry'] / (10 * current_week))
selections.columns = ['Player_ID','Player','Position','Selections', 'Selected_Percent']
selections.head()

In [ ]:
weekly_selections = squads[['element','event','entry']].groupby(['element','event']).agg({'entry':'count'}).sort_values(['event','entry'],ascending=[True,False]).reset_index()
weekly_selections['selection_percent'] = round(100 * weekly_selections['entry'] / 10)
weekly_selections.head()

In [ ]:
weekly_selections

In [ ]:
weekly_selections[['selection_percent','element']].groupby('selection_percent').count().sort_values('selection_percent',ascending=False).reset_index()

## Weekly Player Performance

### Defenders

In [ ]:
a = player_stats[player_stats.total_points>0][['id','total_points']]

a['blank'] = np.where(a['total_points']<=3, 1, 0)
a['return'] = np.where(((a['total_points']>3)&(a['total_points']<=9)),1,0)
a['haul'] = np.where(a['total_points']>9,1,0)
a = a.groupby('id').agg({'total_points':'sum','blank':'sum','return':'sum','haul':'sum'}).sort_values(['haul','return','blank'], ascending=[False,False,True]).reset_index()
a.head()

In [ ]:
b = pd.merge(
      players[['id','first_name','second_name','web_name','team','element_type','now_cost','ict_index','expected_goal_involvements']],
      positions[['id','singular_name']],
      how='inner',
      left_on='element_type',
      right_on='id'
).drop(columns=['element_type','id_y'])
# b = players[['id','first_name','second_name','web_name','team','element_type','now_cost','ict_index','expected_goal_involvements']]
b.rename(columns={'id_x':'id'},inplace=True)
b.head()

In [ ]:
c = pd.merge(
  a,
  b,
  how='inner',
  on='id'
)
c.head()

In [ ]:
c[c.singular_name=='Defender'][['web_name','now_cost','total_points','haul','return','expected_goal_involvements','ict_index']].head(20)

In [ ]:
c[c.singular_name=='Midfielder'][['web_name','now_cost','total_points','haul','return','expected_goal_involvements','ict_index']].head(20)

In [ ]:
c[c.singular_name=='Forward'][['web_name','now_cost','total_points','haul','return','expected_goal_involvements','ict_index']].head(20)

## League Owned

In [ ]:
players[players.total_points > 0][['id','web_name','first_name','second_name','now_cost','total_points','expected_goal_involvements','expected_goals','goals_scored','assists','ict_index']].sort_values('total_points',ascending=False).head()

In [ ]:
squads.head()

In [ ]:
squads['player_count'] = squads.groupby(['event','element'])['entry'].transform('nunique')

In [ ]:
squads[squads.element==355]

In [ ]:
data = pd.merge(
    squads[['event','element','player_count']].groupby(['event','element']).max().reset_index(),
    players[['id','web_name','first_name','second_name','total_points']],
    left_on='element',
    right_on='id',
    how='inner'
).sort_values(['event','player_count'], ascending=[True,False])

In [ ]:
data

In [ ]:
# unique players per week
data[['event','element']].groupby('event').nunique().sort_values(['event','element'], ascending=[True,False]).reset_index()

In [ ]:
data[data.event==current_week][['event','web_name','player_count','total_points']]